In [1]:
using ITensors
using CUDA
include("TTN_utilities.jl")
using .TTN_utilities

# Basic Introduction

In [2]:
#= # Example of building a simple TTN manually
nsites = 4
sites = [Index(2, "site $i") for i in 1:nsites] =#

In ITensors when we define an index, we are implicitly defining a dimension for a generic vector space. In this case we are definig 4 bidimensional spaces (for example each one can be associated with a spin).

We can use indexes to define and work with tensors. For example defining: 

-- ITensor(index(2,a)) creates a rank-1 tensor of dimension 2, so a vector of lenght 2.

-- ITensor(index(2,a), index*(2,a)) creates a 2x2 matrix, so a transformation between the two 2d spaces


Contraction is a sum over indexes with same name.

In [3]:
#= # Leaf tensors
leaf1 = randomITensor(sites[1], sites[2])
leaf2 = randomITensor(sites[3], sites[4])

# Parent tensor
parent = randomITensor(sites[1], sites[2], sites[3], sites[4])

# contracting
result = parent * leaf1 * leaf2

println("Result of contracting the TTN with random vectors: ", result) =#

In [4]:
#= ABC = @visualize leaf1 * leaf2 * parent edge_labels=(tags=true,); =#

Contracting parent, a ITensor(i1,i2,i3,i4) with the two leaves, ITensor(i1,i2) and ITensor(i3,i4) returns a scalar since we are summing over all indexes.

# Tree-TensorNetwork

Following https://journals.aps.org/prb/abstract/10.1103/PhysRevB.99.155131

### Definition

<img src="./images/diagram_TTN.png" alt="Example Image" width="500"/>

As illustrated in Fig. 1(b), each circle represents
a tensor; each edge of the circle represents an individual
index of the tensor. The first tensor is a matrix connecting the
second and third tensors, while the remaining tensors are all
three-order tensors with three indices. The index between two
tensors is called a virtual bond, which would be contracted
hereafter. The left and right indices of the tensors in the
bottom of the TTN are respectively connected to two pixels
of the input image and hence are called physical bonds.

The bond dimension in a tensor network refers to the dimension of the shared index (or bond) connecting two tensors. 
Higher bond dimensions allow more complex entanglement between subsystems to be represented. In quantum many-body systems, the bond dimension represents the number of states used to approximate the entanglement structure.

Since bond dimension represents on how many indices we are summing, it is related to how well we can represent states with our tensor network.
In the context of Matrix Product States (MPS), the bond dimension directly corresponds to the Schmidt rank of the state for each bipartition. If the bond dimension is h , the MPS can capture states with a maximum Schmidt rank of h.

$|\psi\rangle=\sum_{i=1}^D \lambda_i\left|u_i\right\rangle_A \otimes\left|v_i\right\rangle_B$, where D is the Schmidt rank, the number of non-zero $\lambda$, which quantifies the enganglement between the two subsystems A and B (for D=1 the state is separable).

In [5]:
# let's make a TTN with 3 layers, consisting of rank-3 tensors with bond dimension 2 with random elements

# Define physical indices
n_physical_indexes = 8
physical_indexes = [Index(2, "site $i") for i in 1:n_physical_indexes]

# Define bond indices for each layer counting from the bottom
n_bond_indexes_layer1 = div(n_physical_indexes, 2) # 8 bond indexes
n_bond_indexes_layer2 = div(n_bond_indexes_layer1, 2)   # 4 bond indexes

# Define bond indices
bond_dimension = 2
bond_indexes_layer1 = [Index(bond_dimension, "layer1_bond $i") for i in 1:n_bond_indexes_layer1]
bond_indexes_layer2 = [Index(bond_dimension, "layer2_bond $i") for i in 1:n_bond_indexes_layer2]

# Define tensors and normalize them
layer1_tensors = [ randomITensor(physical_indexes[2*i-1], physical_indexes[2*i], bond_indexes_layer1[i]) for i in 1:n_bond_indexes_layer1 ]
layer1_tensors = [ normalize!(t) for t in layer1_tensors ]
layer2_tensors = [ randomITensor(bond_indexes_layer1[2*i-1], bond_indexes_layer1[2*i], bond_indexes_layer2[i]) for i in 1:n_bond_indexes_layer2 ]
layer2_tensors = [ normalize!(t) for t in layer2_tensors ]
root_tensor = randomITensor(bond_indexes_layer2[1], bond_indexes_layer2[2])
root_tensor = normalize!(root_tensor)

ITensor ord=2 (dim=2|id=977|"layer2_bond1") (dim=2|id=431|"layer2_bond2")
NDTensors.Dense{Float64, Vector{Float64}}

In [6]:
# Converting tensors to CUDA tensors
layer1_tensors = [ cu(t) for t in layer1_tensors ]
layer2_tensors = [ cu(t) for t in layer2_tensors ]
root_tensor = cu(root_tensor)
println("TTN converted to CUDA tensors")

TTN converted to CUDA tensors


In [7]:
# Contracting the TTN as a check of indexes naming
result = root_tensor
for t in layer2_tensors
    result = result * t
end
for t in layer1_tensors
    result = result * t
end

In [8]:
# Make a vector of all layers
TTN = vcat(layer1_tensors, layer2_tensors, [root_tensor])
print(squared_norm(TTN))

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
1.0000001192092896

### Canonicalization basics

To simplify things, we need to bring our tensors in canonical form, in which each tensor is in canonical form for an index (in following case for index $\alpha_2$)
$$
\sum_{\alpha_4, \alpha_5} T_{\alpha_2, \alpha_4, \alpha_5}^{[2]} T_{\alpha_2^{\prime}, \alpha_4, \alpha_5}^{[2]}=\delta_{\alpha_2, \alpha_2^{\prime}},
$$

We can do so, by using QR decomposition. Starting from the leaves, we can decompose each tensor in a Unitary part and a Residual part. Then propagating the residual part towards the root we can for a "central tensor" containing all non-canonical information.

In [ ]:
#= # QR decomposition of a tensor as example
# Example of building a simple TTN manually
nsites = 3
sites = [Index(2, "site $i") for i in 1:nsites]

A = randomITensor(sites[1], sites[2], sites[3])
A = A / norm(A)
Q, R = qr(A, [sites[1], sites[2]], [sites[3]])
println("norma di Q: ", norm(Q))
println("norma di R: ", norm(R)) =#

norma di Q: 1.4142135623730951
norma di R: 1.0


In [ ]:
#= A_reconstructed = Q * R
println("Reconstructed tensor: ", A_reconstructed)
println("Reconstruction error: ", norm(A - A_reconstructed))
 =#

Reconstructed tensor: ITensor ord=3
Dim 1: (dim=2|id=94|"site1")
Dim 2: (dim=2|id=475|"site2")
Dim 3: (dim=2|id=880|"site3")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
  0.6369867870151104   0.3969227295013099
 -0.34269229258817324  0.19266989172896

[:, :, 2] =
 -0.4463347575876417   0.01497464111510948
  0.2836623309589739  -0.04729917211522243
Reconstruction error: 8.457555311126093e-17


In [ ]:
#= # Check that Q is unitary
# Normalize Q
Q = Q
Q_dagger = dag(Q)
result = Q * Q_dagger
println("Q * Q_dagger: ", result) =#

Q * Q_dagger: ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
2.0


### Canonicalization of TTN

We can now procede in canonicalization procedure for each layer of the TNN, taking the root tensor as central tensor. In this procedure we have to keep an eye on indexes, since the QR decomposition, the first block of indices is assigned to Q and the second one (in our case, the only "upper" one) is assigned to R, to be contracted with the next layer. The algorithm creates a new set of indexes beetween the first and second layer, that we have to rename.

In [13]:
# Saving indexes
old_indexes_layer1 = [inds(A) for A in layer1_tensors]
old_indexes_layer2 = [inds(A) for A in layer2_tensors]
old_indexes_root = [inds(root_tensor)]

1-element Vector{Tuple{Index{Int64}, Index{Int64}}}:
 ((dim=2|id=977|"layer2_bond1"), (dim=2|id=431|"layer2_bond2"))

In [14]:
# Canonicalization algorithm:
# osservazione: tutti i nodi dell'albero sono normalizzati, sto normalizzando tutti i Q e spostando la norma su R

println("Starting canonical conversion...")
println("Canonical conversion of layer 1")
for i in 1:length(layer1_tensors)
    A = layer1_tensors[i]
    Q, R = qr(A, [physical_indexes[2*i-1], physical_indexes[2*i]], [bond_indexes_layer1[i]])
    Q = Q / norm(Q)
    R = R * norm(Q)
    layer1_tensors[i] = Q
    layer2_tensors[Int(ceil(i/2))] = layer2_tensors[Int(ceil(i/2))] * R
    println("decomposed tensor $i in layer 1")
end

Starting canonical conversion...
Canonical conversion of layer 1
decomposed tensor 1 in layer 1
decomposed tensor 2 in layer 1
decomposed tensor 3 in layer 1
decomposed tensor 4 in layer 1


In [15]:
# Renaming indexes
println("Renaming indexes")
rename_tensor_indices!(layer1_tensors, old_indexes_layer1)
rename_tensor_indices!(layer2_tensors, old_indexes_layer2)

Renaming indexes


In [16]:
println("Canonical conversion of layer 2")
for i in 1:length(layer2_tensors)
    A = layer2_tensors[i]
    Q, R = qr(A, [bond_indexes_layer1[i*2-1], bond_indexes_layer1[2*i]], [bond_indexes_layer2[i]])
    Q = Q / norm(Q)
    R = R * norm(Q)
    layer2_tensors[i] = Q
    root_tensor = root_tensor * R
    println("decomposed tensor $i in layer 2")
end

Canonical conversion of layer 2
decomposed tensor 1 in layer 2
decomposed tensor 2 in layer 2


In [17]:
# Rename indexes of layer 2 tensors
rename_tensor_indices!(layer2_tensors, old_indexes_layer2)
rename_tensor_indices!(root_tensor, old_indexes_root)

In [18]:
#= println("Layer 1:")
print_tensor_vector(layer1_tensors, "Layer 1 Tensors")
println("Layer 2:")
print_tensor_vector(layer2_tensors, "Layer 2 Tensors")
println("Root:")
print_tensor_vector(root_tensor, "Root Tensor") =#

Let's check the squared norm of the entire TTN making it contract with its conjugate (function defined in TTN_utilities)

In [ ]:
# Make a vector of all layers
TTN = vcat(layer1_tensors, layer2_tensors, [root_tensor])
println(squared_norm(TTN))

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.009564132429659061

In [ ]:
println(root_tensor*dag(root_tensor))

ITensor ord=0
NDTensors.Dense{Float32, CuArray{Float32, 1, CUDA.DeviceMemory}}
 0-dimensional
0.009564132f0

As we wanted, the root is now the central tensor, and its squared norm equals to the one of the entire TTN.